# Tutorial 1: Getting Started with SwarmRL

This small tutorial will walkthrough some important componants of the SwarmRL library and how it can be used. In this lesson you will learn the following:

* Installing SwarmRL
* Importing SwarmRL into your project
* Running a simple simulation

Once you are finished with this tutorial you will be able to run a small simulation using the library and look at the results.

## Imports

The first thing you need to do is import the library. For this tutorial, we will also be important the espresso engine seperately to save time later as well as a few helper libraries for looking at the finished simulation.

In [ ]:
import swarmrl as srl
import swarmrl.engine.espresso as espresso
import pint

import numpy as np
import matplotlib.pyplot as plt
import h5py as hf

import znvis as vis

## Building the simulation

Now we can build up the system we want to simulate. This requires the following steps:

* Define the unit registry
* Define some simulation parameters
* Build an EspressoMD object

Important to note in SwarmRL is the difference between the time step and the time slice. The time step is the integrator step, i.e, how far in time you jump in the integrator. The time slice is how often the external force model is used to move the colloids. These should rarely be the same time and are often bound by an experiment. In the real world, your time step is a infinitesimally small number and the time slice might be 10 seconds.

In [ ]:
ureg = pint.UnitRegistry()

In [ ]:
md_params = espresso.MDParams(
            ureg=ureg,
            fluid_dyn_viscosity=ureg.Quantity(8.9e-4, "pascal * second"),
            WCA_epsilon=ureg.Quantity(293, "kelvin") * ureg.boltzmann_constant,
            temperature=ureg.Quantity(293, "kelvin"),
            box_length=ureg.Quantity(1000, "micrometer"),
            time_slice=ureg.Quantity(0.2, "second"),  # model timestep
            time_step=ureg.Quantity(0.2, "second") / 5,  # integrator timestep
            write_interval=ureg.Quantity(2, "second"),
        )

In [ ]:
system_runner = espresso.EspressoMD(
            md_params=md_params,
            n_dims=2,
            seed=42,  # seed for the simulation velocities
            out_folder="swarmrl-getting-started",
            write_chunk_size=1000,  # Used for dumping to the database.
        )

## Adding colloids

Right now you have an empty simulation box with some fluid properties. Now we need to add in the particles. In this case, to show off what SwarmRL can do, we will add particles of three different species (or types) to the simulation.

In [ ]:
system_runner.add_colloids(
            n_colloids=10,  # Let's make 10 of them
            radius_colloid=ureg.Quantity(1.0, "micrometer"),
            random_placement_center=ureg.Quantity(
                np.array([500, 500, 0]), "micrometer"
            ),
            random_placement_radius=ureg.Quantity(60, "micrometer"),
            type_colloid=0,  # These ones are type 0
        )

In [ ]:
system_runner.add_colloids(
            n_colloids=10,  # Let's make 10 of them
            radius_colloid=ureg.Quantity(2.0, "micrometer"),
            random_placement_center=ureg.Quantity(
                np.array([500, 500, 0]), "micrometer"
            ),
            random_placement_radius=ureg.Quantity(60, "micrometer"),
            type_colloid=1,  # These ones are type 1
        )

In [ ]:
system_runner.add_colloids(
            n_colloids=10,  # Let's make 10 of them
            radius_colloid=ureg.Quantity(3.0, "micrometer"),
            random_placement_center=ureg.Quantity(
                np.array([500, 500, 0]), "micrometer"
            ),
            random_placement_radius=ureg.Quantity(60, "micrometer"),
            type_colloid=2,  # These ones are type 2
        )

## Running the simulation

Now we can run the simulation for a few timesteps by simply calling integrate. Note that it is at this stage that an external force model could be added to the simulation to apply a force other than Brownian motion.

In [ ]:
system_runner.integrate(n_slices=10000)

## Analysing the simulation

Now we can look at the simulation data we have generated. Here we will perform the following:

1. Use matplotlib to visualize trajectories
2. Use the ZnVis library to visualize them in 3D

The first thing we need to do is load the data using H5PY as follows.

In [ ]:
with hf.File("swarmrl-getting-started/trajectory.hdf5", 'r') as db:
    position_data = db["colloids"]["Unwrapped_Positions"][:]
    type_data = db["colloids"]["Types"][:]

The type data shows us that for all time, the first 10 points are colloid 1, from 10 to 20 are colloid 2, and the rest are colloid 3. So let's split the data up for convenience.

In [ ]:
colloid_1 = position_data[:, :10, :]
colloid_2 = position_data[:, 10:20, :]
colloid_3 = position_data[:, 20:, :]

Now we can use matplotlib to visualize the trajectory.

In [ ]:
# Trailing lines
plt.plot(colloid_1[:, :, 0], colloid_1[:, :, 1], 'r')
plt.plot(colloid_2[:, :, 0], colloid_2[:, :, 1], 'g')
plt.plot(colloid_3[:, :, 0], colloid_3[:, :, 1], 'b')

# Final positions
plt.plot(colloid_1[0, :, 0], colloid_1[0, :, 1], 'ko', markersize=10)
plt.plot(colloid_2[0, :, 0], colloid_2[0, :, 1], 'ko', markersize=12)
plt.plot(colloid_3[0, :, 0], colloid_3[0, :, 1], 'ko', markersize=15)

plt.show()

We have scaled up the marker sizes according to the simulation choice. We can see here a lot of randomness due to the Brownian motion.

Now let's use a particle visualizer to watch this in 3D.

In [ ]:
# Define colloid 1
mesh = vis.Sphere(radius=1.0, colour=np.array([1, 25, 54]) / 255, resolution=10)
colloid_1_model = vis.Particle(name="colloid_1", mesh=mesh, position=colloid_1)

# Define colloid 2
mesh = vis.Sphere(radius=2.0, colour=np.array([249, 220, 92]) / 255, resolution=10)
colloid_2_model = vis.Particle(name="colloid_2", mesh=mesh, position=colloid_2)

# Define colloid 3
mesh = vis.Sphere(radius=3.0, colour=np.array([237, 37, 78]) / 255, resolution=10)
colloid_3_model = vis.Particle(name="colloid_3", mesh=mesh, position=colloid_3)

In [ ]:
visualizer = vis.Visualizer(
    particles=[colloid_1_model, colloid_2_model, colloid_3_model], 
    frame_rate=20
)
visualizer.run_visualization()

You should see a seperate window pop up from which you can start watching the trajectory as well as save screenshots and export scenes to `.obj` files for additional rendering. 

## Summing up

You have made it to the end of tutorial 1 on SwarmRL. We hope it has been informative and a good introduction to the very foundations of the package. Next up we show how to add a classical model to these simulations and start to see some collective behaviour.